In [5]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from importlib import reload
from tqdm import tqdm
import data_utils
import pandas as pd
import pytorch_tabr
import numpy as np
from sklearn.model_selection import StratifiedKFold, train_test_split
from xgboost import XGBClassifier
from typing import Tuple
from tqdm import tqdm

pd.set_option('display.max_columns', 70)
pd.set_option('display.max_rows', 70)

SEED = 42
NUM_SPLITS = 10
TARGET = "decision"

In [2]:
reload(data_utils)
from data_utils import get_dataset

X, y = get_dataset()

/home/krzaq/xai/XAI-team-project/data_utils.py:31: DtypeWarning: Columns (4,11,12,16,17,18,19,20,40,41,42,43,44,45,52,53,54,55,56,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,108,110) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv("SpeedDating.csv", index_col=0)


In [3]:
X.describe()

,gender,age,age_o,importance_same_race,importance_same_religion,pref_o_attractive,pref_o_sincere,pref_o_intelligence,pref_o_funny,pref_o_ambitious,pref_o_shared_interests,attractive_o,sinsere_o,intelligence_o,funny_o,ambitous_o,shared_interests_o,attractive_important,sincere_important,intellicence_important,funny_important,ambtition_important,shared_interests_important,attractive,sincere,intelligence,funny,ambition,attractive_partner,sincere_partner,intelligence_partner,funny_partner,ambition_partner,shared_interests_partner,sports,tvsports,exercise,dining,museums,art,hiking,gaming,clubbing,reading,tv,theater,movies,concerts,music,shopping,yoga,interests_correlate,expected_happy_with_sd_people,expected_num_matches,expected_num_interested_in_me,like,guess_prob_liked,race_?,race_Asian/Pacific Islander/Asian-American,race_Black/African American,race_European/Caucasian-American,race_Latino/Hispanic American,race_Other,race_o_?,race_o_Asian/Pacific Islander/Asian-American,race_o_Black/African American,race_o_European/Caucasian-American,race_o_Latino/Hispanic American,race_o_Other
count,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000,8378.000000
mean,0.499403,26.358928,26.364999,3.784793,3.651645,22.495347,17.396867,20.270759,17.459714,10.685375,11.845930,6.190411,7.175256,7.369301,6.400599,6.778409,5.474870,22.514632,17.396389,20.265613,17.457043,10.682539,11.845111,7.084733,8.294935,7.704460,8.403965,7.578388,6.189995,7.175164,7.368597,6.400598,6.777524,5.474559,6.425232,4.575491,6.245813,7.783829,6.985781,6.714544,5.737077,3.881191,5.745993,7.678515,5.304133,6.776118,7.919629,6.825401,7.851066,5.631281,4.339197,0.196010,5.534131,3.207814,5.570556,6.134087,5.207523,0.007520,0.236572,0.050131,0.564216,0.079255,0.062306,0.008713,0.236095,0.050131,0.563619,0.079255,0.062187
std,0.500029,3.546480,3.541458,2.832258,2.791978,12.502851,7.006484,6.746767,6.049825,6.087291,6.313565,1.925469,1.710499,1.521919,1.911629,1.715024,2.012929,12.528179,7.013394,6.750943,6.052827,6.088588,6.316038,1.387008,1.398611,1.554486,1.069839,1.767135,1.926513,1.711300,1.522814,1.912453,1.716119,2.014356,2.606645,2.788632,2.407425,1.746573,2.042532,2.252709,2.558059,2.608122,2.490391,1.997082,2.517181,2.224588,1.692888,2.146091,1.783358,2.596582,2.704767,0.300663,1.723574,2.267192,2.207052,1.814717,2.089919,0.086395,0.425003,0.218229,0.495889,0.270153,0.241725,0.092943,0.424706,0.218229,0.495966,0.270153,0.241509
min,0.000000,18.000000,18.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,2.000000,2.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,-0.830000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,24.000000,24.000000,1.000000,1.000000,15.000000,15.000000,17.650000,15.000000,5.000000,10.000000,5.000000,6.000000,7.000000,5.000000,6.000000,4.000000,15.000000,15.00

In [7]:
reload(pytorch_tabr)
from pytorch_tabr import TabRClassifier
# Train and test the model
kf = StratifiedKFold(n_splits=NUM_SPLITS, shuffle=True, random_state=SEED)

tabr_classifier = TabRClassifier(
    type_embeddings="one-hot",
    device_name="cuda",
    optimizer_params={"lr": 2e-4},
    d_main=96,
    context_size=96,
    # selection_function_name="sparsemax",
    # context_dropout=0.5,
    # context_sample_size=2000,
    # num_embeddings={"type": "PLREmbeddings", "n_frequencies": 32, "frequency_scale": 32, "d_embedding": 32, "lite": False},
)
s=0
for train_idx, test_idx in kf.split(X, y):
    X_train, y_train = X.iloc[train_idx, :], y.iloc[train_idx]
    X_test, y_test = X.iloc[test_idx, :], y.iloc[test_idx]
    tabr_classifier.fit(X_train.values, y_train.values, max_epochs=2, batch_size=10)
    print('Split accuracy: ', np.mean(tabr_classifier.predict(X_test).values == np.array(y_test)))
    s += np.mean(tabr_classifier.predict(X_test) == np.array(y_test))
    print('Accuracy for class 1 [person wanted to match]', np.sum( np.logical_and(tabr_classifier.predict(X_test) == 1, tabr_classifier.predict(X_test) == y_test))/np.sum(y_test))
print('Average accuracy:')
print(s/NUM_SPLITS)

/home/krzaq/xai/XAI-team-project/pytorch_tabr/base_model.py:258: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


 epochs:   0%|          | 0/2 [00:00<?, ?it/s]

 batches:   0%|          | 0/754 [00:00<?, ?it/s]

/home/krzaq/.local/share/mamba/envs/tabr/lib/python3.9/site-packages/faiss/contrib/torch_utils.py:51: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  x.storage().data_ptr() + x.storage_offset() * 4)
/home/krzaq/.local/share/mamba/envs/tabr/lib/python3.9/site-packages/faiss/contrib/torch_utils.py:65: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  x.storage().data_ptr() + x.storage_offset() * 8)


 batches:   0%|          | 0/754 [00:00<?, ?it/s]

KeyError: 0